In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import Pool, CatBoostRegressor
%matplotlib inline


/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [51]:
train_data = pd.read_csv('data/train_agg_2.csv', index_col = 0)

In [68]:
test_data = pd.read_csv('data/test.csv', index_col = 0)

In [20]:
set(train_data.columns.tolist()).difference(test_data.columns.tolist())

{'ID', 'РегионУчастника'}

In [54]:
train_data['СуммаГарантииПредложенияЛота'][train_data['СуммаГарантииПредложенияЛота'] == '-'] = 0

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [55]:

train_label = train_data['ID']
train_clean = train_data.drop(['IDOrganizator', 'РегионУчастника', 'ID'], axis = 1)


In [66]:
list(zip(train_clean.columns, range(len(train_clean.columns))))

[('Минимальная глубина CPV (2017 год)', 0),
 ('СуммаТендера', 1),
 ('Организатор', 2),
 ('Дата начала подачи предложений', 3),
 ('Тип методу закупівлі', 4),
 ('Тип организатора', 5),
 ('Статус лота', 6),
 ('СтатусТендера', 7),
 ('РегионОрганизатора', 8),
 ('Регион поставки', 9),
 ('Тендер', 10),
 ('Дата начала периода уточнений', 11),
 ('Дата завершения', 12),
 ('Дата завершения подачи предложений', 13),
 ('ШагУменьшенияЛот', 14),
 ('СуммаГарантииПредложенияЛота', 15)]

In [59]:
cats = list(range(2, 14))

In [72]:
test_clean = test_data.drop(['IDOrganizator'], axis = 1)

In [73]:
test_clean['СуммаГарантииПредложенияЛота'][test_clean['СуммаГарантииПредложенияЛота'] == '-'] = 0

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [70]:
list(zip(test_clean.columns, range(len(test_clean.columns))))

[('Минимальная глубина CPV (2017 год)', 0),
 ('СуммаТендера', 1),
 ('Организатор', 2),
 ('Дата начала подачи предложений', 3),
 ('Тип методу закупівлі', 4),
 ('Тип организатора', 5),
 ('Статус лота', 6),
 ('СтатусТендера', 7),
 ('РегионОрганизатора', 8),
 ('Регион поставки', 9),
 ('Тендер', 10),
 ('Дата начала периода уточнений', 11),
 ('Дата завершения', 12),
 ('Дата завершения подачи предложений', 13),
 ('ШагУменьшенияЛот', 14),
 ('СуммаГарантииПредложенияЛота', 15)]

In [74]:
train_pool = Pool(train_clean, train_label, cat_features=cats)
test_pool = Pool(test_clean, cat_features=cats)

In [77]:
model = CatBoostRegressor(iterations=5000, depth=5, learning_rate=0.1, loss_function="RMSE")

In [78]:
model.fit(train_pool)

0:	learn: 2.1645535	total: 349ms	remaining: 29m 4s
1:	learn: 2.0206553	total: 600ms	remaining: 24m 58s
2:	learn: 1.8960155	total: 731ms	remaining: 20m 16s
3:	learn: 1.7886415	total: 986ms	remaining: 20m 31s
4:	learn: 1.6966633	total: 1.13s	remaining: 18m 45s
5:	learn: 1.6182561	total: 1.19s	remaining: 16m 33s
6:	learn: 1.5514140	total: 1.31s	remaining: 15m 34s
7:	learn: 1.4951140	total: 1.46s	remaining: 15m 11s
8:	learn: 1.4478283	total: 1.64s	remaining: 15m 7s
9:	learn: 1.4077241	total: 1.84s	remaining: 15m 16s
10:	learn: 1.3701737	total: 2.05s	remaining: 15m 31s
11:	learn: 1.3388853	total: 2.24s	remaining: 15m 29s
12:	learn: 1.3128532	total: 2.4s	remaining: 15m 19s
13:	learn: 1.2935335	total: 2.52s	remaining: 14m 56s
14:	learn: 1.2766765	total: 2.71s	remaining: 15m
15:	learn: 1.2627409	total: 2.82s	remaining: 14m 39s
16:	learn: 1.2513235	total: 2.94s	remaining: 14m 21s
17:	learn: 1.2419369	total: 3.07s	remaining: 14m 9s
18:	learn: 1.2317920	total: 3.19s	remaining: 13m 57s
19:	learn: 

156:	learn: 1.1336553	total: 33.6s	remaining: 17m 16s
157:	learn: 1.1335882	total: 33.8s	remaining: 17m 16s
158:	learn: 1.1334409	total: 34s	remaining: 17m 16s
159:	learn: 1.1331280	total: 34.2s	remaining: 17m 15s
160:	learn: 1.1329503	total: 34.5s	remaining: 17m 16s
161:	learn: 1.1328183	total: 34.7s	remaining: 17m 16s
162:	learn: 1.1326231	total: 34.9s	remaining: 17m 16s
163:	learn: 1.1325201	total: 35.2s	remaining: 17m 17s
164:	learn: 1.1324125	total: 35.5s	remaining: 17m 19s
165:	learn: 1.1323442	total: 35.7s	remaining: 17m 19s
166:	learn: 1.1321568	total: 35.9s	remaining: 17m 17s
167:	learn: 1.1320420	total: 36s	remaining: 17m 16s
168:	learn: 1.1319259	total: 36.3s	remaining: 17m 17s
169:	learn: 1.1318740	total: 36.6s	remaining: 17m 19s
170:	learn: 1.1318268	total: 36.8s	remaining: 17m 18s
171:	learn: 1.1317430	total: 37.1s	remaining: 17m 20s
172:	learn: 1.1316637	total: 37.3s	remaining: 17m 20s
173:	learn: 1.1315870	total: 37.6s	remaining: 17m 21s
174:	learn: 1.1314770	total: 37.

309:	learn: 1.1193850	total: 1m 9s	remaining: 17m 28s
310:	learn: 1.1193541	total: 1m 9s	remaining: 17m 29s
311:	learn: 1.1193338	total: 1m 9s	remaining: 17m 29s
312:	learn: 1.1192795	total: 1m 10s	remaining: 17m 28s
313:	learn: 1.1192475	total: 1m 10s	remaining: 17m 29s
314:	learn: 1.1192408	total: 1m 10s	remaining: 17m 29s
315:	learn: 1.1191312	total: 1m 10s	remaining: 17m 29s
316:	learn: 1.1191056	total: 1m 10s	remaining: 17m 27s
317:	learn: 1.1190658	total: 1m 11s	remaining: 17m 28s
318:	learn: 1.1189740	total: 1m 11s	remaining: 17m 28s
319:	learn: 1.1188997	total: 1m 11s	remaining: 17m 27s
320:	learn: 1.1188184	total: 1m 11s	remaining: 17m 28s
321:	learn: 1.1187779	total: 1m 12s	remaining: 17m 28s
322:	learn: 1.1187391	total: 1m 12s	remaining: 17m 29s
323:	learn: 1.1186735	total: 1m 12s	remaining: 17m 29s
324:	learn: 1.1186202	total: 1m 12s	remaining: 17m 29s
325:	learn: 1.1186051	total: 1m 13s	remaining: 17m 28s
326:	learn: 1.1185519	total: 1m 13s	remaining: 17m 29s
327:	learn: 1

459:	learn: 1.1124717	total: 1m 45s	remaining: 17m 21s
460:	learn: 1.1124475	total: 1m 45s	remaining: 17m 21s
461:	learn: 1.1124260	total: 1m 45s	remaining: 17m 21s
462:	learn: 1.1124192	total: 1m 46s	remaining: 17m 19s
463:	learn: 1.1123914	total: 1m 46s	remaining: 17m 19s
464:	learn: 1.1123253	total: 1m 46s	remaining: 17m 19s
465:	learn: 1.1122754	total: 1m 46s	remaining: 17m 18s
466:	learn: 1.1122581	total: 1m 46s	remaining: 17m 18s
467:	learn: 1.1121743	total: 1m 47s	remaining: 17m 18s
468:	learn: 1.1120903	total: 1m 47s	remaining: 17m 18s
469:	learn: 1.1120873	total: 1m 47s	remaining: 17m 17s
470:	learn: 1.1120696	total: 1m 47s	remaining: 17m 17s
471:	learn: 1.1120248	total: 1m 48s	remaining: 17m 17s
472:	learn: 1.1120089	total: 1m 48s	remaining: 17m 16s
473:	learn: 1.1119641	total: 1m 48s	remaining: 17m 16s
474:	learn: 1.1119559	total: 1m 48s	remaining: 17m 17s
475:	learn: 1.1118861	total: 1m 49s	remaining: 17m 16s
476:	learn: 1.1118744	total: 1m 49s	remaining: 17m 16s
477:	learn

610:	learn: 1.1076130	total: 2m 20s	remaining: 16m 51s
611:	learn: 1.1075863	total: 2m 21s	remaining: 16m 51s
612:	learn: 1.1075632	total: 2m 21s	remaining: 16m 51s
613:	learn: 1.1075108	total: 2m 21s	remaining: 16m 50s
614:	learn: 1.1074982	total: 2m 21s	remaining: 16m 50s
615:	learn: 1.1074742	total: 2m 22s	remaining: 16m 50s
616:	learn: 1.1074511	total: 2m 22s	remaining: 16m 50s
617:	learn: 1.1074028	total: 2m 22s	remaining: 16m 50s
618:	learn: 1.1073948	total: 2m 22s	remaining: 16m 49s
619:	learn: 1.1073519	total: 2m 22s	remaining: 16m 49s
620:	learn: 1.1073380	total: 2m 23s	remaining: 16m 49s
621:	learn: 1.1073316	total: 2m 23s	remaining: 16m 49s
622:	learn: 1.1073106	total: 2m 23s	remaining: 16m 49s
623:	learn: 1.1072856	total: 2m 23s	remaining: 16m 49s
624:	learn: 1.1072702	total: 2m 24s	remaining: 16m 49s
625:	learn: 1.1072440	total: 2m 24s	remaining: 16m 49s
626:	learn: 1.1071848	total: 2m 24s	remaining: 16m 49s
627:	learn: 1.1071625	total: 2m 24s	remaining: 16m 48s
628:	learn

...

4319:	learn: 1.0761949	total: 17m 22s	remaining: 2m 44s
4320:	learn: 1.0761942	total: 17m 22s	remaining: 2m 43s
4321:	learn: 1.0761932	total: 17m 22s	remaining: 2m 43s
4322:	learn: 1.0761836	total: 17m 23s	remaining: 2m 43s
4323:	learn: 1.0761704	total: 17m 23s	remaining: 2m 43s
4324:	learn: 1.0761694	total: 17m 23s	remaining: 2m 42s
4325:	learn: 1.0761603	total: 17m 23s	remaining: 2m 42s
4326:	learn: 1.0761533	total: 17m 24s	remaining: 2m 42s
4327:	learn: 1.0761519	total: 17m 24s	remaining: 2m 42s
4328:	learn: 1.0761494	total: 17m 24s	remaining: 2m 41s
4329:	learn: 1.0761454	total: 17m 24s	remaining: 2m 41s
4330:	learn: 1.0761442	total: 17m 25s	remaining: 2m 41s
4331:	learn: 1.0761432	total: 17m 25s	remaining: 2m 41s
4332:	learn: 1.0761418	total: 17m 25s	remaining: 2m 40s
4333:	learn: 1.0761336	total: 17m 25s	remaining: 2m 40s
4334:	learn: 1.0761204	total: 17m 26s	remaining: 2m 40s
4335:	learn: 1.0761133	total: 17m 26s	remaining: 2m 40s
4336:	learn: 1.0761106	total: 17m 26s	remaining:

4466:	learn: 1.0755093	total: 17m 58s	remaining: 2m 8s
4467:	learn: 1.0755053	total: 17m 58s	remaining: 2m 8s
4468:	learn: 1.0754972	total: 17m 58s	remaining: 2m 8s
4469:	learn: 1.0754931	total: 17m 58s	remaining: 2m 7s
4470:	learn: 1.0754919	total: 17m 59s	remaining: 2m 7s
4471:	learn: 1.0754913	total: 17m 59s	remaining: 2m 7s
4472:	learn: 1.0754880	total: 17m 59s	remaining: 2m 7s
4473:	learn: 1.0754853	total: 18m	remaining: 2m 7s
4474:	learn: 1.0754847	total: 18m	remaining: 2m 6s
4475:	learn: 1.0754836	total: 18m	remaining: 2m 6s
4476:	learn: 1.0754825	total: 18m	remaining: 2m 6s
4477:	learn: 1.0754807	total: 18m 1s	remaining: 2m 6s
4478:	learn: 1.0754789	total: 18m 1s	remaining: 2m 5s
4479:	learn: 1.0754771	total: 18m 1s	remaining: 2m 5s
4480:	learn: 1.0754661	total: 18m 1s	remaining: 2m 5s
4481:	learn: 1.0754642	total: 18m 2s	remaining: 2m 5s
4482:	learn: 1.0754630	total: 18m 2s	remaining: 2m 4s
4483:	learn: 1.0754595	total: 18m 2s	remaining: 2m 4s
4484:	learn: 1.0754531	total: 18m

4615:	learn: 1.0748539	total: 18m 34s	remaining: 1m 32s
4616:	learn: 1.0748390	total: 18m 34s	remaining: 1m 32s
4617:	learn: 1.0748294	total: 18m 35s	remaining: 1m 32s
4618:	learn: 1.0748247	total: 18m 35s	remaining: 1m 32s
4619:	learn: 1.0748089	total: 18m 35s	remaining: 1m 31s
4620:	learn: 1.0748080	total: 18m 35s	remaining: 1m 31s
4621:	learn: 1.0748064	total: 18m 36s	remaining: 1m 31s
4622:	learn: 1.0748046	total: 18m 36s	remaining: 1m 31s
4623:	learn: 1.0748040	total: 18m 36s	remaining: 1m 30s
4624:	learn: 1.0748035	total: 18m 37s	remaining: 1m 30s
4625:	learn: 1.0747924	total: 18m 37s	remaining: 1m 30s
4626:	learn: 1.0747848	total: 18m 37s	remaining: 1m 30s
4627:	learn: 1.0747767	total: 18m 37s	remaining: 1m 29s
4628:	learn: 1.0747757	total: 18m 37s	remaining: 1m 29s
4629:	learn: 1.0747739	total: 18m 38s	remaining: 1m 29s
4630:	learn: 1.0747726	total: 18m 38s	remaining: 1m 29s
4631:	learn: 1.0747712	total: 18m 38s	remaining: 1m 28s
4632:	learn: 1.0747683	total: 18m 38s	remaining:

4764:	learn: 1.0743034	total: 19m 11s	remaining: 56.8s
4765:	learn: 1.0743006	total: 19m 11s	remaining: 56.6s
4766:	learn: 1.0742999	total: 19m 12s	remaining: 56.3s
4767:	learn: 1.0742992	total: 19m 12s	remaining: 56.1s
4768:	learn: 1.0742968	total: 19m 12s	remaining: 55.8s
4769:	learn: 1.0742918	total: 19m 12s	remaining: 55.6s
4770:	learn: 1.0742855	total: 19m 13s	remaining: 55.4s
4771:	learn: 1.0742731	total: 19m 13s	remaining: 55.1s
4772:	learn: 1.0742706	total: 19m 13s	remaining: 54.9s
4773:	learn: 1.0742691	total: 19m 14s	remaining: 54.6s
4774:	learn: 1.0742666	total: 19m 14s	remaining: 54.4s
4775:	learn: 1.0742606	total: 19m 14s	remaining: 54.2s
4776:	learn: 1.0742586	total: 19m 14s	remaining: 53.9s
4777:	learn: 1.0742554	total: 19m 15s	remaining: 53.7s
4778:	learn: 1.0742266	total: 19m 15s	remaining: 53.4s
4779:	learn: 1.0742255	total: 19m 15s	remaining: 53.2s
4780:	learn: 1.0742238	total: 19m 15s	remaining: 52.9s
4781:	learn: 1.0742192	total: 19m 16s	remaining: 52.7s
4782:	lear

4914:	learn: 1.0736617	total: 19m 48s	remaining: 20.6s
4915:	learn: 1.0736590	total: 19m 49s	remaining: 20.3s
4916:	learn: 1.0736589	total: 19m 49s	remaining: 20.1s
4917:	learn: 1.0736545	total: 19m 49s	remaining: 19.8s
4918:	learn: 1.0736388	total: 19m 49s	remaining: 19.6s
4919:	learn: 1.0736305	total: 19m 49s	remaining: 19.3s
4920:	learn: 1.0736260	total: 19m 50s	remaining: 19.1s
4921:	learn: 1.0736236	total: 19m 50s	remaining: 18.9s
4922:	learn: 1.0736215	total: 19m 50s	remaining: 18.6s
4923:	learn: 1.0736182	total: 19m 50s	remaining: 18.4s
4924:	learn: 1.0736153	total: 19m 51s	remaining: 18.1s
4925:	learn: 1.0736132	total: 19m 51s	remaining: 17.9s
4926:	learn: 1.0736108	total: 19m 51s	remaining: 17.7s
4927:	learn: 1.0736068	total: 19m 51s	remaining: 17.4s
4928:	learn: 1.0736023	total: 19m 51s	remaining: 17.2s
4929:	learn: 1.0736018	total: 19m 52s	remaining: 16.9s
4930:	learn: 1.0736007	total: 19m 52s	remaining: 16.7s
4931:	learn: 1.0736003	total: 19m 52s	remaining: 16.4s
4932:	lear

In [12]:
from sklearn.metrics import mean_squared_error

In [95]:
y_p = model.predict(test_pool)

In [99]:
train_clean.to_csv('~/data/train_s4.csv')

In [100]:
test_clean.to_csv('~/data/test_clean.csv')

In [86]:
y_p

array([1.        , 1.17112018, 1.        , ..., 2.23597381, 2.72567036,
       2.72567036])

In [97]:
pd.DataFrame(list(zip(test_clean.index, y_p))).to_csv('~/results/submission_4.csv', index=False, header=['unique_id', 'target'])